In [1]:
import sys

import numpy as np
import skimage as ski
from cellpose import models
import torch
import nd2

import napari

In [2]:
viewer = napari.Viewer()

In [3]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='cyto2')

### nd2 files
We will be using a Nikon file for this notebook, using the `nd2` package. The `nd2` package has an imread function like skimage and tifffile:

```python
data = nd2.imread(filename)
```

Instead, we will use the `ND2File` object gives us access to the metadata (like x, y, z scaling, excitation wavelengths, etc.) that we will need.


In [4]:
imagefile = 'files/cellpose/Data/WT003.nd2'

### create the file object - this doesn't read the image array data
nd2file = nd2.ND2File(imagefile)
image = nd2file.asarray()
image.shape

(40, 3, 2044, 2048)

Add the image to the napari viewer. From the shape output in the last cell, the channel axis is 1

In [5]:
viewer.add_image(image, channel_axis=1)

[<Image layer 'Image' at 0x19953e06440>,
 <Image layer 'Image [1]' at 0x19953e1dbd0>,
 <Image layer 'Image [2]' at 0x19954dcaa40>]

Rotate the image in 3D and notice the 3D scaling is not set. To get the right values for the z scaling we need to look at the metadata.


### Metadata

Exlpore what is inside the metadata variable of the nd2file object

In [6]:
nd2file.metadata

Metadata(contents=Contents(channelCount=3, frameCount=40), channels=[Channel(channel=ChannelMeta(name='488 sCMOS', index=0, colorRGB=65280, emissionLambdaNm=535.0, excitationLambdaNm=None), loops=LoopIndices(NETimeLoop=None, TimeLoop=None, XYPosLoop=None, ZStackLoop=0), microscope=Microscope(objectiveMagnification=100.0, objectiveName='Plan Apo λ 100x Oil', objectiveNumericalAperture=1.45, zoomMagnification=1.0, immersionRefractiveIndex=1.515, projectiveMagnification=None, pinholeDiameterUm=50.0, modalityFlags=['fluorescence', 'spinningDiskConfocal']), volume=Volume(axesCalibrated=(True, True, True), axesCalibration=(0.065, 0.065, 0.3), axesInterpretation=('distance', 'distance', 'distance'), bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, cameraTransformationMatrix=(0.999993283193413, -0.003665183223043356, 0.003665183223043356, 0.999993283193413), componentCount=1, componentDataType='unsigned', voxelCount=(2048, 2044, 40), componentMaxima=[0.0], componentMinima=[0.0], pi

In [7]:
md1 = nd2file.metadata.channels[1]
xum, yum, zum = md1.volume.axesCalibration
xum, yum, zum

(0.065, 0.065, 0.3)

Now let's show the image again, but properly scaled to microns

In [8]:
viewer.layers.clear()
viewer.add_image(image, channel_axis=1, name='image scaled', scale=(zum, yum, xum))

[<Image layer 'image scaled' at 0x19949296980>,
 <Image layer 'image scaled [1]' at 0x19935308940>,
 <Image layer 'image scaled [2]' at 0x19935321a80>]

Our XY resolution is roughly 5 times better than our Z resolution, we will want to know what this ratio is exactly for resampling in the next section so let's calculate it.

In [9]:
zscale = xum/zum
zscale

0.21666666666666667

### Cellpose 3D

Cellpose has a 3D option in `model.eval`, but it is not really 3D. It does the eval on every xy plane, then every yz plane, then every xz plane. After the eval cellpose reconstructs the results from each plane into 3D masks/labels.  For this to work the scaling in z needs to match the xy scaling. The `anisotropy` parameter can be used, and cellpose will adjust the input image, but in practice this does not work well. Another option is to rescale the image in XY to have the same (lower) resolution that Z has before using cellpose. 

### Scaling down the image

In [10]:
scaled = ski.transform.rescale(image, (1, 1, zscale, zscale), preserve_range=True)
scaled = ski.filters.gaussian(scaled, sigma=(1, 0, 2, 2))
scaled.shape

(40, 3, 443, 444)

In [11]:
viewer.layers.clear()
viewer.add_image(scaled, channel_axis=1)

[<Image layer 'Image' at 0x1993501a6e0>,
 <Image layer 'Image [1]' at 0x1997fc9ab00>,
 <Image layer 'Image [2]' at 0x1997fc621a0>]

### Running cellpose 3D

In [12]:
masks, _, _, _ = model.eval(scaled, diameter=75, do_3D=True, channels=[1,2],
                            cellprob_threshold=1,
                            flow_threshold=.3) 

In [13]:
viewer.add_labels(masks)

<Labels layer 'masks' at 0x1993fd56d70>

## Scaling back up the cellpose result

To use the cellpose results with the original image, the masks need to be scaled back to the original size. The `order` parameter is the key to making this successful.
Settting `order=0` makes `resize` use nearest neighbors when upscaling an image rather than interpolation. 

In [14]:
shape = (image.shape[0], image.shape[2], image.shape[3])
smasks = ski.transform.resize(masks, shape, order=0, preserve_range=True)

In [15]:
viewer.layers.clear()
viewer.add_image(image, channel_axis=1, scale=(zum, yum, xum))
viewer.add_labels(smasks, scale=(zum, yum, xum))


<Labels layer 'smasks' at 0x19954df63b0>